# 07 — Benchmark Comparisons
Compare ARIMA-only and ARIMA+LSTM against simple correlation baselines.

In [ ]:
import numpy as np
import pandas as pd

corr_df = pd.read_parquet("../data/processed/rolling_corr_sample.parquet")

In [ ]:
# Baselines
def baseline_full_historical(train, test):
    mu = float(np.mean(train))
    return np.full_like(test, mu, dtype="float32")

def baseline_constant_correlation(train, test):
    # same as mean baseline (kept separate for clarity)
    return baseline_full_historical(train, test)

def baseline_last_value(train, test):
    last = float(train[-1])
    return np.full_like(test, last, dtype="float32")

def metrics(y_true, y_hat):
    y_true = y_true.astype("float32")
    y_hat = y_hat.astype("float32")
    mse = float(np.mean((y_true - y_hat)**2))
    mae = float(np.mean(np.abs(y_true - y_hat)))
    return mse, mae

In [ ]:
# Evaluate baselines on one series
series = corr_df.columns[0]
y = corr_df[series].dropna().values.astype("float32")

n = len(y)
train_end = int(0.7*n)
val_end = int(0.85*n)

train = y[:train_end]
test  = y[val_end:]

pred_mean = baseline_full_historical(train, test)
pred_last = baseline_last_value(train, test)

print("Mean baseline:", metrics(test, pred_mean))
print("Last-value baseline:", metrics(test, pred_last))

In [ ]:
# Table template for final reporting
rows = []
rows.append({"model": "Mean (Full Historical)", "mse": metrics(test, pred_mean)[0], "mae": metrics(test, pred_mean)[1]})
rows.append({"model": "Last Value", "mse": metrics(test, pred_last)[0], "mae": metrics(test, pred_last)[1]})

pd.DataFrame(rows).sort_values("mse")